In [1]:
import yfinance as yf
# import argparse
import atexit
import datetime
import logging
import schedule
import time
import json

In [2]:
logger_format = '%(asctime)-15s %(message)s'
logging.basicConfig(format=logger_format)
logger = logging.getLogger('test')
logger.setLevel(logging.DEBUG)

In [3]:
def log_exception(msg,exception):
  logger.warning(f'{msg}, caused by, {exception}')

In [4]:
from kafka import KafkaProducer
from kafka.errors import KafkaError, KafkaTimeoutError

In [5]:
topic_name = 'stockTopic'
kafka_broker = '127.0.0.1:9092'

In [6]:
def fetch_symbol_price(symbol,producer):
  '''
  function to fetch price of the stock and push it to the producer instance
  of kafka
  :param symbol: symbol of the stock
  :param producer: instance of the kafka producer
  :return: None
  '''

  logger.info(f'Fetching stock price of {symbol}')
  try:
#     print(yf.Ticker(symbol).info)
    price=yf.Ticker(symbol).info['ask']
    timestamp=datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%dT%H:%M:%SZ')
    payload={
        'StockSymbol':symbol,
        'Price':price,
        'Time':timestamp
    }
    producer.send(topic=topic_name, value=payload)
    logger.info(f'Received stock price of {symbol} at {timestamp} : {price}')
  except KafkaError as ke:
    logger.warning(f'Kafka Error {ke}')
  except Exception as e:
    logger.warning(f'Failed to fetch stock price of {symbol} caused by {e}')
  return payload

In [7]:
def on_shutdown(producer):
  '''
  shutdown hook to be called before the shutdown
  :param producer: instance of a kafka producer
  :return: None
  '''
  try:
    logger.info('Flushing pending messages to kafka, timeout is set to 10s')
    producer.flush(10) 
    logger.info('Finish flushing pending messages to kafka')
  finally:
      try:
        logger.info('Closing the Kafka connection')
        producer.close(10)
      except Exception as e:
        log_exception('Failed to close kafka connection',e)

In [8]:
if __name__=='__main__':
    symbol='GC=F'
    producer=KafkaProducer(
        bootstrap_servers=kafka_broker,
        value_serializer=lambda x: json.dumps(x).encode('utf-8')
    )
    schedule.every(1).second.do(fetch_symbol_price,symbol,producer)
    atexit.register(on_shutdown,producer)
    while True:
        schedule.run_pending()
        time.sleep(1)

2021-11-25 19:28:00,395 Fetching stock price of GC=F
2021-11-25 19:28:06,293 Received stock price of GC=F at 2021-11-25T19:28:06Z : 1792.4
2021-11-25 19:28:07,299 Fetching stock price of GC=F
2021-11-25 19:28:12,400 Received stock price of GC=F at 2021-11-25T19:28:12Z : 1792.4
2021-11-25 19:28:13,410 Fetching stock price of GC=F
2021-11-25 19:28:19,542 Received stock price of GC=F at 2021-11-25T19:28:19Z : 1792.4
2021-11-25 19:28:20,556 Fetching stock price of GC=F
2021-11-25 19:28:25,798 Received stock price of GC=F at 2021-11-25T19:28:25Z : 1792.4
2021-11-25 19:28:26,813 Fetching stock price of GC=F
2021-11-25 19:28:32,296 Received stock price of GC=F at 2021-11-25T19:28:32Z : 1792.4
2021-11-25 19:28:33,308 Fetching stock price of GC=F
2021-11-25 19:28:38,709 Received stock price of GC=F at 2021-11-25T19:28:38Z : 1792.4
2021-11-25 19:28:39,718 Fetching stock price of GC=F
2021-11-25 19:28:44,888 Received stock price of GC=F at 2021-11-25T19:28:44Z : 1792.4
2021-11-25 19:28:45,897 Fet

KeyboardInterrupt: 